In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
import os
import scipy as sp
import scipy.fftpack
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]
from sklearn.preprocessing import StandardScaler


In [8]:
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\global_hypha_infos.pick"
global_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\time_hypha_infos.pick"
time_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\AMOLF-SHIMIZU Dropbox\Corentin Bisot\Applications\princesync\data_tables\global_plate_infos.pick"
global_plate_info = pd.read_pickle(path_save)

In [91]:
time_plate_info.loc[time_plate_info['Plate']==799]["datetime"]

44    2021-10-01 16:20:00
45    2021-10-01 18:20:00
46    2021-10-01 20:20:00
47    2021-10-01 22:20:00
48    2021-10-02 00:20:00
              ...        
134   2021-10-11 04:27:00
135   2021-10-11 06:27:00
136   2021-10-11 08:27:00
137   2021-10-11 10:27:00
138   2021-10-11 12:27:00
Name: datetime, Length: 95, dtype: datetime64[ns]

In [90]:
list(time_plate_info.loc[time_plate_info['Plate']==799].columns)

['t',
 'num_trunks',
 'area',
 'area_sep_comp',
 'num_tips',
 'num_nodes',
 'area_study',
 'num_tips_study',
 'num_nodes_study',
 'tot_length_study',
 'out_study',
 'date',
 'Plate',
 'path_exp',
 'folder_analysis',
 'ring_density_incr-100_index-0',
 'ring_density_incr-100_index-1',
 'ring_density_incr-100_index-2',
 'ring_density_incr-100_index-3',
 'ring_density_incr-100_index-4',
 'ring_density_incr-100_index-5',
 'ring_density_incr-100_index-6',
 'ring_density_incr-100_index-7',
 'ring_density_incr-100_index-8',
 'ring_density_incr-100_index-9',
 'ring_density_incr-100_index-10',
 'ring_density_incr-100_index-11',
 'ring_density_incr-100_index-12',
 'ring_density_incr-100_index-13',
 'ring_density_incr-100_index-14',
 'ring_density_incr-100_index-15',
 'ring_density_incr-100_index-16',
 'ring_density_incr-100_index-17',
 'ring_density_incr-100_index-18',
 'ring_density_incr-100_index-19',
 'ring_bas_density_incr-100_index-0',
 'ring_bas_density_incr-100_index-1',
 'ring_bas_density

In [10]:
plates = time_plate_info['Plate'].unique()
plates = [94,792,76,152,102]
ind_plate_table= {}
for plate in plates:
    ind_plate_table[plate]=time_plate_info.loc[time_plate_info['Plate']==plate].copy()
time_hypha_table = {}
for plate in plates: 
    select_time_plate = ind_plate_table[plate]
    last_time = np.max(select_time_plate.loc[select_time_plate['out_study']==0]['time_since_begin'])
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate].copy()
    select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
    select_time = select_time.loc[select_time['distance_final_pos']>200]

    select_time['speed'] = select_time['speed'].astype(float)
    time_hypha_table[plate] = select_time
roundin = 1000
ind_hypha_table = {}
for plate in plates:
    table =global_hypha_info.loc[global_hypha_info['Plate']==plate].copy()
    table = table.set_index('hypha')
    ind_hypha_table[plate] = table
for plate in plates:
    table = ind_hypha_table[plate].copy()
    table_time = time_hypha_table[plate].copy()
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate].copy()
    speed_at_init = table_time.loc[table_time['time_since_emergence']==0]
    table['max_speed'] = table_time.groupby('end')['speed'].max()
    table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
    table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
    table['growth_lapse'] = table_time.groupby('end')['speed'].count()
    table['speed_at_init'] = speed_at_init.groupby('end')['speed'].max()
    table['branch_frequ'] = table['num_branch']/(table['tot_growth_C']+1)
for plate in plates:
    table = ind_plate_table[plate]
    table['total_length_m'] = table['tot_length_study']/10**6
    table['speed'] = table['tot_length_study'].diff(5)/table['time_since_begin'].diff(5)
    table['mean_step'] = np.mean(table['time_since_begin'].diff(1))
    table['density'] = table['tot_length_study']/(table['area_sep_comp']+1)
    table['ring_density'] = table['tot_length_study'].diff(10)/(table['area_sep_comp'].diff(10)+1)
    table['log_length_study'] = np.log2(table['tot_length_study'].astype(float))
    table['local_growth_rate'] = np.gradient(table['log_length_study'],table['time_since_begin'])*24
    table_hypha = ind_hypha_table[plate]

In [11]:
select_plates = [94,792,76,152,102]

t0s = {94 : 22,102 : 32,152:27,792 : 44,76:13}
t1s = {94 : 46,102 : 60,152:51,792 : 61,76:28}

In [12]:
select_plates = [94,102,152,792,76]

fig, ax = plt.subplots()
ax2=ax.twinx()
x = 'time_since_begin3'
y = 'total_length_m'
logy =False
# ylim = (100,300)
xlim = (-40,100)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network length $( m)$'
num_columns = 2
total = pd.DataFrame()
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["area_sep_comp"]>=800]["time_since_begin"])
    print(t0,plate)
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0+40
    table['time_since_begin4'] = table['time_since_begin']-t1s[plate]
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate,alpha=0.5)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    ax.set_yscale('log')
    y = 'total_length_m'
    f = interp1d(in_study[x], in_study[y],bounds_error=False,fill_value=None)
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    deb = 1
    end = 90
    frequ = 1
    av_lapse = 311
    x0 = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x0,'total_length_m':f(x0)})
#     reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
#     resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 2)
#     resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
#     resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24
#     ax2.plot(resampled['time'],resampled['local_growth_rate'],label = plate)
    data = resampled[['time','total_length_m']]
    total = pd.concat((total,data))
# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  
ax2.set_ylim((0,3))  
ax2.set_ylabel('relative growth rate (.day-1)',size = 20)
plt.legend()
# ax2.vlines(0,0,3,color="black")

data = total.groupby('time')['total_length_m'].mean()
ax.plot(data,color="red",linewidth=5)
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


55.15 94
83.2 102
100.06666666666666 152
82.18333333333334 792
56.016666666666666 76


Text(0.04, 0.5, 'Total network length $( m)$')

In [27]:
select_plates = [94,102,152,792,76]

fig, ax = plt.subplots()
# ax2=ax.twinx()
x = 'time_since_begin3'
y = 'ratio'
logy =False
# ylim = (100,300)
xlim = (-40,100)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'BAS_length/total_length$( m)$'
num_columns = 2
total = pd.DataFrame()
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    table['ratio'] = table['L_BAS']/table['total_length_m']/10**6
    t0 = np.min(table.loc[table["area_sep_comp"]>=800]["time_since_begin"])
    print(t0,plate)
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0+40
    table['time_since_begin4'] = table['time_since_begin']-t1s[plate]
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate,alpha=0.5)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
#     ax.set_yscale('log')
    f = interp1d(in_study[x], in_study[y],bounds_error=False,fill_value=None)
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    deb = 1
    end = 90
    frequ = 1
    av_lapse = 311
    x0 = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x0,'total_length_m':f(x0)})
#     reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
#     resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 2)
#     resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
#     resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24
#     ax2.plot(resampled['time'],resampled['local_growth_rate'],label = plate)
    data = resampled[['time','total_length_m']]
    total = pd.concat((total,data))
# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  
# ax2.set_ylim((0,3))  
# ax2.set_ylabel('relative growth rate (.day-1)',size = 20)
plt.legend()
# ax2.vlines(0,0,3,color="black")

data = total.groupby('time')['total_length_m'].mean()
ax.plot(data,color="red",linewidth=5)
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

55.15 94
83.2 102
100.06666666666666 152
82.18333333333334 792
56.016666666666666 76


Text(0.04, 0.5, 'BAS_length/total_length$( m)$')

In [15]:
list(table.columns)

['t',
 'num_trunks',
 'area',
 'area_sep_comp',
 'num_tips',
 'num_nodes',
 'area_study',
 'num_tips_study',
 'num_nodes_study',
 'tot_length_study',
 'out_study',
 'date',
 'Plate',
 'path_exp',
 'folder_analysis',
 'ring_density_incr-100_index-0',
 'ring_density_incr-100_index-1',
 'ring_density_incr-100_index-2',
 'ring_density_incr-100_index-3',
 'ring_density_incr-100_index-4',
 'ring_density_incr-100_index-5',
 'ring_density_incr-100_index-6',
 'ring_density_incr-100_index-7',
 'ring_density_incr-100_index-8',
 'ring_density_incr-100_index-9',
 'ring_density_incr-100_index-10',
 'ring_density_incr-100_index-11',
 'ring_density_incr-100_index-12',
 'ring_density_incr-100_index-13',
 'ring_density_incr-100_index-14',
 'ring_density_incr-100_index-15',
 'ring_density_incr-100_index-16',
 'ring_density_incr-100_index-17',
 'ring_density_incr-100_index-18',
 'ring_density_incr-100_index-19',
 'ring_bas_density_incr-100_index-0',
 'ring_bas_density_incr-100_index-1',
 'ring_bas_density

In [8]:
total

,0,log_total_length_smooth,time
0,-46.000000,NaN,NaN
1,-45.919653,NaN,NaN
2,-45.839306,NaN,NaN
3,-45.758959,NaN,NaN
4,-45.678612,NaN,NaN
...,...,...,...
995,0.901162,NaN,NaN
996,0.900500,NaN,NaN
997,0.899837,NaN,NaN
998,0.899173,NaN,NaN


In [9]:
select_plates = [152]
select_plates = [94,792,76,152,102]

fig, ax = plt.subplots()
ax2=ax.twinx()
x = 'time_since_begin4'
y = 'total_length_m'
y2 = 'derivative'
logy =False
# ylim = (100,300)
xlim = (-20,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network area $(mm^2)$'
tau = 50
num_columns = 2
delta = 200
av_lapse = 211

frequ = 0.1
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    table['time_since_begin4'] = table['time_since_begin']-t1s[plate]+tau
    in_study = table.loc[table['out_study']==0]
#     if len(in_study)>0:
#         ax.scatter(in_study[x],in_study[y],label = plate)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
#     ax.set_yscale('log')
#     ax.set_xscale('log')

    f = interp1d(in_study[x], in_study[y])
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    x0 = np.linspace(deb,end,int((end-deb)/frequ))
    resampled = pd.DataFrame({'time':x0,y:f(x0)})
    reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
    resampled['log_total_length']=np.log2(resampled[y])

    resampled['log_total_length_m_smooth']= savgol_filter(resampled['log_total_length'], av_lapse, 3)
#     resampled['log_time']=np.log2(resampled['time'])
    resampled['time2']=resampled['time']

#     resampled['log_total_length_m_smooth'] = resampled['log_total_length_m_smooth']-np.log(np.min(table.loc[table['time_since_begin']>=t1s[plate]][y]))
#     resampled['derivative'] = resampled[y].diff(delta)/resampled['time'].diff(delta)/(np.pi/2*0.3**2)
    resampled['derivative'] = np.gradient(resampled[y],resampled['time'])
    resampled['derivative'] = savgol_filter(resampled['derivative'], av_lapse, 1)
    resampled['2ndderivative'] = np.gradient(resampled['derivative'],resampled['time'])
#     resampled['2ndderivative'] = savgol_filter(resampled['2ndderivative'], av_lapse, 1)
    ax2.vlines(tau-t0s[plate],0,0.20,color='black')

#     resampled['2ndderivative'] = resampled['derivative'].diff(delta)/resampled['time'].diff(delta)

#     ax2.scatter(resampled['time'],resampled['local_growth_rate'],label = plate)
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate)
    ax2.plot(resampled['time2'],resampled[y2],label = plate)
sat_density = 1000
incr = -0.03
ax2.plot(resampled['time2'],2*resampled['time2']/sat_density+incr,label = "y=2x/sat_density+beta",linestyle = ':')

# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  

# ax2.vlines(tau,0,0.20,color='black')
# ax2.set_ylim((0,25))  
ax2.set_ylim((0,0.05))  
ax2.set_ylim(None)  

ax2.set_ylabel('derivative rescaled by $\pi / 2 *v^2$',size = 20)
plt.legend()
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Total network area $(mm^2)$')

In [6]:
select_plates = [152]
select_plates = [94,792,76,152,102]

fig, ax = plt.subplots()
ax2=ax.twinx()
x = 'time_since_begin4'
y = 'total_length_m'
y2 = 'derivative'
logy =False
# ylim = (100,300)
xlim = (-20,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network area $(mm^2)$'
tau = 50
num_columns = 2
delta = 200
av_lapse = 211

frequ = 0.1
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    table['time_since_begin4'] = table['time_since_begin']-t1s[plate]+tau
    in_study = table.loc[table['out_study']==0]
#     if len(in_study)>0:
#         ax.scatter(in_study[x],in_study[y],label = plate)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
#     ax.set_yscale('log')
#     ax.set_xscale('log')

    f = interp1d(in_study[x], in_study[y])
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    x0 = np.linspace(deb,end,int((end-deb)/frequ))
    resampled = pd.DataFrame({'time':x0,y:f(x0)})
    reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
    resampled['log_total_length']=np.log2(resampled[y])

    resampled['log_total_length_m_smooth']= savgol_filter(resampled['log_total_length'], av_lapse, 3)
#     resampled['log_time']=np.log2(resampled['time'])
    resampled['time2']=resampled['time']

#     resampled['log_total_length_m_smooth'] = resampled['log_total_length_m_smooth']-np.log(np.min(table.loc[table['time_since_begin']>=t1s[plate]][y]))
#     resampled['derivative'] = resampled[y].diff(delta)/resampled['time'].diff(delta)/(np.pi/2*0.3**2)
    resampled['derivative'] = np.gradient(resampled[y],resampled['time'])/(np.pi/2*0.3**2)
    resampled['derivative'] = savgol_filter(resampled['derivative'], av_lapse, 1)
    resampled['2ndderivative'] = np.gradient(resampled['derivative'],resampled['time'])
#     resampled['2ndderivative'] = savgol_filter(resampled['2ndderivative'], av_lapse, 1)
    ax2.vlines(tau-t0s[plate],0,0.20,color='black')

#     resampled['2ndderivative'] = resampled['derivative'].diff(delta)/resampled['time'].diff(delta)

#     ax2.scatter(resampled['time'],resampled['local_growth_rate'],label = plate)
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate)
    ax2.plot(resampled['time2'],resampled[y2],label = plate)
sat_density = 1000
incr = -0.03
ax2.plot(resampled['time2'],2*resampled['time2']/sat_density+incr,label = "y=2x/sat_density+beta",linestyle = ':')

# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  

ax2.vlines(tau,0,0.20,color='black')
# ax2.set_ylim((0,25))  
# ax2.set_ylim((0,3))  
ax2.set_ylim(None)  

ax2.set_ylabel('derivative rescaled by $\pi / 2 *v^2$',size = 20)
plt.legend()
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Total network area $(mm^2)$')

In [144]:
x = 'time_since_begin3'
y = 'area_sep_comp'
fig, ax = plt.subplots()
ax2=ax.twinx()

delta = -60
av_lapse = 11
data = pd.DataFrame()
for i,plate in enumerate(select_plates):    
    table = ind_plate_table[plate]
    y0 = np.min(table.loc[table['time_since_begin']>=t1s[plate]+delta][y])
#     table = table.loc[table['time_since_begin']>=t1s[plate]+delta].copy()
    table[y] = table[y].astype(float)
#     t0 = np.min(table.loc[table["total_length_m"]>=0.5]["time_since_begin"])
    t0 = np.min(table.loc[table["area_sep_comp"]>=800]["time_since_begin"])
    table = table.loc[table['out_study']==0].copy()

    table['renorm'] = table[y]-y0+1
    table['log_renorm'] = np.log(table["renorm"])
    table['time_since_begin3'] = table['time_since_begin']-t0+90
    
    table['time_since_begin4'] = table['time_since_begin']-(t1s[plate]+delta)+1
#     table['log_time'] = np.log(table["time_since_begin4"])
#     table['derivative'] = np.gradient(table["log_renorm"],table['log_time'])
#     table['derivative'] = savgol_filter(table['derivative'], av_lapse, 1)

#     table['derivative']
    ax.scatter(table['time_since_begin3'],table[y],label = plate)
#     ax2.plot(table['time_since_begin4'],table['derivative'],label = plate)
    ax.set_yscale('log')
    ax.set_xscale('log')
    data = resampled[['time','total_length_m']]
    total = pd.concat((total,data))
ax.set_xlim(10)
ax.set_ylim(0)
# ax2.set_ylim(0,4)
ax.set_xlabel('time (h)')
ax.set_ylabel('network area $(\mu m^2)$')
data = total.groupby('time')['total_length_m'].mean()
ax.plot(data,color="red",linewidth=5)
# ax2.set_ylabel('slope of the log-log')
# ax2.vlines(-delta,0,4,color= 'black')
# ts = np.linspace(10,1000,1000)
# ax.plot(ts,(ts+20)**2-10000)
# ax.plot(table['log_time'],2*table['log_time'])
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
No handles with labels found to put in legend.
